In [1]:
import xpress as xp
import numpy as np
import pandas as pd
import time


In [2]:
# Load distance matrix
distance_matrix = np.loadtxt('distance_matrix50.csv', delimiter=',')
stations = pd.read_csv('station_data.csv')
pois = pd.read_csv('edinburgh_pois.csv')

In [3]:
I = distance_matrix.shape[0]  # Number of POIs (demand points)
J = distance_matrix.shape[1] # Number of candidate stations

In [4]:
# IMPORTANT: Set the number of facilities to locate
p = 15  # Change this to test different scenarios

print(f"\n{'='*70}")
print(f"MODEL PARAMETERS")
print(f"{'='*70}")
print(f"Number of POIs (I):                {I}")
print(f"Number of candidate stations (J):  {J}")
print(f"Number of stations to locate (p):  {p}")


MODEL PARAMETERS
Number of POIs (I):                50
Number of candidate stations (J):  85
Number of stations to locate (p):  15


In [6]:
prob = xp.problem(name='p_centre_bike_stations50')

In [7]:
print("="*70)
print("DEFINE DECISION VARIABLES")
print("="*70)

DEFINE DECISION VARIABLES


In [8]:
# Y[j] = 1 if facility j is opened, 0 otherwise
Y = {j: prob.addVariable(vartype=xp.binary, name=f'Y_{j}') for j in range(J)}

# X[i,j] = 1 if demand point i is assigned to station j, 0 otherwise
X = {(i, j): prob.addVariable(vartype=xp.binary, name=f'X_{i}_{j}') 
     for i in range(I) for j in range(J)}

# Q = maximum service distance (minimax objective)
Q = prob.addVariable(name='Q', lb=0)

print(f"Created {J} Y variables (station selection)")
print(f" Created {I*J:,} X variables (POI-station assignments)")
print(f" Created 1 W variable (minimax distance)")
print(f"  Total variables: {J + I*J + 1:,}")


Created 85 Y variables (station selection)
 Created 4,250 X variables (POI-station assignments)
 Created 1 W variable (minimax distance)
  Total variables: 4,336


In [9]:
print("="*70)
print("ADD CONSTRAINTS")
print("="*70)

ADD CONSTRAINTS


In [10]:
# CONSTRAINT 1: Each POI must be assigned to exactly one station
print("  Adding constraint 1: POI assignment (each POI assigned to exactly one station)...")
for i in range(I):
    prob.addConstraint(xp.Sum(X[i, j] for j in range(J)) == 1)

# CONSTRAINT 2: Exactly p stations must be opened
print(f"  Adding constraint 2: Facility count (open exactly {p} stations)...")
prob.addConstraint(xp.Sum(Y[j] for j in range(J)) == p)

# CONSTRAINT 3: Assignment only to open stations (linking constraints)
print("  Adding constraint 3: Linking constraints (assign only to open stations)...")
for i in range(I):
    for j in range(J):
        prob.addConstraint(X[i, j] <= Y[j])

# CONSTRAINT 4: Minimax distance constraint
print("  Adding constraint 4: Minimax distance (Q >= distance to assigned station)...")
for i in range(I):
    prob.addConstraint(
        xp.Sum(distance_matrix[i, j] * X[i, j] for j in range(J)) <= Q)

  Adding constraint 1: POI assignment (each POI assigned to exactly one station)...
  Adding constraint 2: Facility count (open exactly 15 stations)...
  Adding constraint 3: Linking constraints (assign only to open stations)...
  Adding constraint 4: Minimax distance (Q >= distance to assigned station)...


In [11]:
print("="*70)
print("SET OBJECTIVE FUNCTION")
print("="*70)

SET OBJECTIVE FUNCTION


In [12]:
# Minimize W (the maximum service distance)
prob.setObjective(Q, sense=xp.minimize)

In [13]:
print("\n" + "="*70)
print("SOLVING MODEL...")
print("="*70 + "\n")

start_solve = time.time()

# Solve the problem
xp.setOutputEnabled(True)
prob.solve()

solve_time = time.time() - start_solve


SOLVING MODEL...

FICO Xpress v9.7.0, Hyper, solve started 15:36:23, Nov 6, 2025
Heap usage: 3031KB (peak 3031KB, 762KB system)
Minimizing MILP p_centre_bike_stations50 using up to 8 threads and up to 8192MB memory, with these control settings:
OUTPUTLOG = 1
NLPPOSTSOLVE = 1
XSLP_DELETIONCONTROL = 0
XSLP_OBJSENSE = 1
Original problem has:
      4351 rows         4336 cols        17135 elements      4335 entities
Presolved problem has:
      4351 rows         4336 cols        16238 elements      4335 entities
Presolve finished in 0 seconds
Heap usage: 5518KB (peak 7583KB, 762KB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 2.25e-02,  1.74e+01] / [ 9.61e-05,  2.00e+00]
  RHS and bounds [min,max] : [ 1.00e+00,  1.50e+01] / [ 2.17e-01,  1.58e+02]
  Objective      [min,max] : [ 1.00e+00,  1.00e+00] / [ 4.00e+00,  4.00e+00]
Autoscaling applied standard scaling

Will try to keep branch and bound tree memory usage below 7.

In [14]:
    print("\n" + "="*70)
    print("SOLUTION SUMMARY")
    print("="*70)
    

    # Extract optimal minimax distance
    Q_optimal = prob.getSolution(Q)
    
    # Extract which stations to open
    stations_open = []
    for j in range(J):
        if prob.getSolution(Y[j]) > 0.5:  # Binary variable, so > 0.5 means 1
            stations_open.append(j)
    



SOLUTION SUMMARY


In [15]:
    print(f"\n{'='*70}")
    print(f"OPTIMAL P-CENTRE RESULTS")
    print(f"{'='*70}")
    
    print(f"\n1. STATIONS TO OPEN ({len(stations_open)} out of {J}):")
    print(f"   Station indices: {stations_open}")
    
    print(f"\n   Station details:")
    optimal_stations_list = []
    for idx, j in enumerate(stations_open, 1):
        station_id = stations.iloc[j]['station_id']
        station_name = stations.iloc[j]['name']
        capacity = stations.iloc[j]['capacity']
        lat = stations.iloc[j]['lat']
        lon = stations.iloc[j]['lon']
        
        optimal_stations_list.append({
            'index': j,
            'rank': idx,
            'station_id': station_id,
            'station_name': station_name,
            'capacity': capacity,
            'latitude': lat,
            'longitude': lon
        })
        
        print(f"   {idx:2d}. [{j:2d}] {station_name:30s} (ID: {station_id}, Capacity: {capacity:2d} docks)")
    
    print(f"\n2. OPTIMAL MINIMAX DISTANCE:")
    print(f"   Maximum service distance (W*) = {Q_optimal:.4f} km")
    print(f"   ✓ All {I:,} POIs served within {Q_optimal:.4f} km of nearest station")


OPTIMAL P-CENTRE RESULTS

1. STATIONS TO OPEN (15 out of 85):
   Station indices: [2, 3, 6, 9, 13, 15, 17, 32, 37, 41, 58, 63, 71, 75, 84]

   Station details:
    1. [ 2] Musselburgh Lidl               (ID: 2263, Capacity: 34 docks)
    2. [ 3] Leith Walk North               (ID: 2259, Capacity: 32 docks)
    3. [ 6] Edinburgh Park Station         (ID: 1822, Capacity: 23 docks)
    4. [ 9] Heriot Watt - Edinburgh Business School (ID: 1819, Capacity: 20 docks)
    5. [13] Milton Road - Edinburgh College (ID: 1813, Capacity: 24 docks)
    6. [15] Gamekeeper's Road              (ID: 1807, Capacity: 20 docks)
    7. [17] Ellersly Road                  (ID: 1770, Capacity: 23 docks)
    8. [32] Edinburgh Royal Infirmary      (ID: 1739, Capacity: 23 docks)
    9. [37] Portobello - Kings Road        (ID: 1728, Capacity: 69 docks)
   10. [41] Cramond Foreshore              (ID: 1722, Capacity: 38 docks)
   11. [58] Dundee Terrace                 (ID: 1025, Capacity: 25 docks)
   12. [63] Orc

In [20]:
    assignments = {}
    for i in range(I):
        for j in range(J):
            if prob.getSolution(X[i, j]) > 0.5:
                assignments[i] = j
                break

In [21]:
    print(f"   All {I:,} POIs assigned: {len(assignments) == I}")
    
    # Count POIs per station
    pois_per_station = {}
    distances_per_station = {}
    
    for i, j in assignments.items():
        if j not in pois_per_station:
            pois_per_station[j] = 0
            distances_per_station[j] = []
        pois_per_station[j] += 1
        distances_per_station[j].append(distance_matrix[i, j])
    
    print(f"\n   POIs assigned per station:")
    for rank, j in enumerate(stations_open, 1):
        count = pois_per_station.get(j, 0)
        avg_dist = np.mean(distances_per_station.get(j, ))
        max_dist = np.max(distances_per_station.get(j, ))
        
        print(f"   {rank:2d}. {stations.iloc[j]['name']:30s}: {count:5,} POIs (avg: {avg_dist:.3f} km, max: {max_dist:.3f} km)")
    

   All 50 POIs assigned: True

   POIs assigned per station:
    1. Musselburgh Lidl              :     2 POIs (avg: 0.883 km, max: 1.509 km)
    2. Leith Walk North              :     5 POIs (avg: 0.974 km, max: 1.480 km)
    3. Edinburgh Park Station        :     2 POIs (avg: 0.998 km, max: 1.436 km)
    4. Heriot Watt - Edinburgh Business School:     1 POIs (avg: 0.240 km, max: 0.240 km)
    5. Milton Road - Edinburgh College:     1 POIs (avg: 0.400 km, max: 0.400 km)
    6. Gamekeeper's Road             :     2 POIs (avg: 1.380 km, max: 1.629 km)
    7. Ellersly Road                 :     2 POIs (avg: 0.977 km, max: 1.252 km)
    8. Edinburgh Royal Infirmary     :     1 POIs (avg: 0.139 km, max: 0.139 km)
    9. Portobello - Kings Road       :     1 POIs (avg: 0.031 km, max: 0.031 km)
   10. Cramond Foreshore             :     2 POIs (avg: 1.518 km, max: 1.732 km)
   11. Dundee Terrace                :     7 POIs (avg: 0.996 km, max: 1.719 km)
   12. Orchard Brae House            :

In [23]:
    all_assigned_distances = [distance_matrix[i, assignments[i]] for i in range(I)]
    
    print(f" DISTANCE STATISTICS:")
    print(f"   Mean distance: {np.mean(all_assigned_distances):.4f} km")
    print(f"   Median distance: {np.median(all_assigned_distances):.4f} km")
    print(f"   Std Dev: {np.std(all_assigned_distances):.4f} km")
    print(f"   Min distance: {np.min(all_assigned_distances):.4f} km")
    print(f"   Max distance (W*): {np.max(all_assigned_distances):.4f} km")
    
    # Coverage analysis
    print(f" COVERAGE ANALYSIS:")
    print(f"   POIs within 0.5 km: {(np.array(all_assigned_distances) < 0.5).sum():,} ({100*(np.array(all_assigned_distances) < 0.5).sum()/I:.2f}%)")
    print(f"   POIs within 1.0 km: {(np.array(all_assigned_distances) < 1.0).sum():,} ({100*(np.array(all_assigned_distances) < 1.0).sum()/I:.2f}%)")
    print(f"   POIs within 2.0 km: {(np.array(all_assigned_distances) < 2.0).sum():,} ({100*(np.array(all_assigned_distances) < 2.0).sum()/I:.2f}%)")
    

 DISTANCE STATISTICS:
   Mean distance: 0.9467 km
   Median distance: 1.0205 km
   Std Dev: 0.5229 km
   Min distance: 0.0313 km
   Max distance (W*): 1.7325 km
 COVERAGE ANALYSIS:
   POIs within 0.5 km: 13 (26.00%)
   POIs within 1.0 km: 25 (50.00%)
   POIs within 2.0 km: 50 (100.00%)
